In [1]:
import requests
import io
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
# import chart_studio
# import chart_studio.plotly as py
import numpy as np

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000
pd.options.mode.chained_assignment = None

In [2]:
# введено из-за того, что аккаунт удалён из общего рейтинга

def resum_column(df, column_name):
    return df[column_name].rolling(min_periods=1, window=GAMEWEEK).sum()

In [3]:
# фокус на нужных значений очков, чтоб можно было бы что-то разглядеть
def range_points(min_x, max_x, padding=50):
    min_y = DATA[DATA.event == min_x].total_points.min() - padding
    max_y = DATA[DATA.event == max_x].total_points.max() + padding
    return [min_y, max_y]

In [4]:
teams = {'1002936': 'Dmitriy', '132645': 'Magnus'}
DATA = pd.DataFrame()

for team_id, name in teams.items():
    
    url = f'https://fantasy.premierleague.com/api/entry/{team_id}/history/'
    r = requests.get(url)
    data_i = pd.DataFrame(r.json()['current'])
    data_i.insert(0, 'FPL Player', name)
    GAMEWEEK = data_i.shape[0]
    # --- добавка в связи с удалением аккаунта ---
    data_i['total_points'] = resum_column(data_i, 'points') - resum_column(data_i, 'event_transfers_cost')
    data_i['total_points'] = data_i['total_points'].astype(int)
    # ------
    DATA = pd.concat([DATA, data_i])

url_standings = 'https://fantasy.premierleague.com/api/leagues-classic/314/standings/?page_standings=1'
r_0 = requests.get(url_standings)
top1 = r_0.json()['standings']['results'][0]
top1_id = top1['entry']
top1_name = top1['player_name']
print(f'Current 1st place: {top1_name}:')
print(f'https://fantasy.premierleague.com/entry/{top1_id}/event/{GAMEWEEK}')

url_top1 = f'https://fantasy.premierleague.com/api/entry/{top1_id}/history/'
r = requests.get(url_top1)
data_top1 = pd.DataFrame(r.json()['current'])
data_top1.insert(0, 'FPL Player', '1st place')
DATA = pd.concat([DATA, data_top1])

Current 1st place: Jamie Pigott:
https://fantasy.premierleague.com/entry/159801/event/38


In [5]:
perfect_points_per_gw = 70
perfect = pd.DataFrame(
    list(zip(range(1,GAMEWEEK+1), [perfect_points_per_gw for i in range(GAMEWEEK)])), 
    columns=['event', 'points']
)

perfect.insert(0, 'FPL Player', 'perfect')
perfect['total_points'] = perfect['points'].rolling(min_periods=1, window=GAMEWEEK).sum()

DATA = pd.concat([DATA, perfect])

In [6]:
# DATA.total_points = DATA.total_points.apply(np.log10)

In [19]:
lst_players = DATA['FPL Player'].unique().tolist()
fig = go.Figure()

for player in lst_players:
    
    slc = DATA[DATA['FPL Player'] == player]
    
    if player == 'perfect':
        fig.add_trace(go.Scatter(x=slc['event'], y=slc['total_points'],
                    mode='lines+markers',
                    name=player,
                    line=dict(dash='dash')))
    else:
        fig.add_trace(go.Scatter(x=slc['event'], y=slc['total_points'],
            mode='lines+markers',
            name=player))     
        
fig.update_xaxes(title_text="Gameweek #", range=[0.1, DATA['event'].max()+1])
fig.update_yaxes(title_text="Total points")

# annotations on the last data points
annotations = sorted(DATA[DATA.event == GAMEWEEK].total_points, reverse=True)
yshifts = [0, 10, -10, -5]
annot_d = dict(zip(annotations, yshifts))

for k, v in annot_d.items():
    fig.add_annotation(
        x=GAMEWEEK, 
        y=k,
        text=str(int(k)),
        showarrow=False,
        # arrowhead=1,
        xshift=25,
        yshift=v,
        font=dict(
            size=12
        )
    )


fig.update_layout(
    title={
        'text': f"Total FPL points comparison (Dmitriy vs. Magnus vs. 1st place vs. perfect trend)",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1,
        range=[GAMEWEEK-9.5, GAMEWEEK+0.5]
    ),
    yaxis=dict(
        range=range_points(GAMEWEEK-10, GAMEWEEK)
    )
)

fig.show()

fig.write_image(
    f"results/images/comparison_FPL_total_points_afterGW{GAMEWEEK}.png", 
    width=1980*0.5, 
    height=1080*0.5
)

In [11]:
sorted(DATA[DATA.event == GAMEWEEK].points, reverse=True)

[72, 70, 69, 62]

In [20]:
fig = go.Figure()

for player in lst_players:
    
    slc = DATA[DATA['FPL Player'] == player]
    
    if player == 'perfect':
        fig.add_trace(go.Scatter(x=slc['event'], y=slc['points'],
                    mode='lines+markers',
                    name=player,
                    line=dict(dash='dash')))
    else:
        fig.add_trace(go.Scatter(x=slc['event'], y=slc['points'],
            mode='lines+markers',
            name=player))
        
        
fig.update_xaxes(title_text="Gameweek #", range=[0.1, DATA['event'].max()+1])
fig.update_yaxes(title_text="Total points")

# annotations on the last data points
annotations = sorted(DATA[DATA.event == GAMEWEEK].points, reverse=True)
yshifts = [20, 10, 0, -10]
annot_d = dict(zip(annotations, yshifts))

for k, v in annot_d.items():
    fig.add_annotation(
        x=GAMEWEEK, 
        y=k,
        text=str(int(k)),
        showarrow=False,
        # arrowhead=1,
        xshift=15,
        yshift=v,
        font=dict(
            size=10
        )
    )


fig.update_layout(
    title={
        'text': f"FPL points per GW comparison (Dmitriy vs. Magnus vs. 1st place vs. perfect trend)",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1,
        range=[0.5, GAMEWEEK+0.5]
    )
)

fig.show()

fig.write_image(f"results/images/comparison_FPL_points_per_week_afterGW{GAMEWEEK}.png", 
                width=1980*0.5, height=1080*0.5)

---
## Deprecated versions

In [175]:
# fig = px.line(DATA, x='event', y='total_points', color='FPL Player').update_traces(mode='lines+markers')

# fig.update_xaxes(title_text="Gameweek #", range=[0.1, data['event'].max()+1])
# fig.update_yaxes(title_text="Total points")

# fig.update_layout(
#     title={
#         'text': f"Total FPL points comparison (Dmitriy vs. Magnus vs. 1st place)",
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'},
#     xaxis = dict(
#         tickmode = 'linear',
#         tick0 = 0,
#         dtick = 1
#     )
# )

# fig.show()

In [176]:
# fig = px.line(DATA, x='event', y='points', color='FPL Player').update_traces(mode='lines+markers')

# fig.update_xaxes(title_text="Gameweek #", range=[0, data['event'].max()+1])
# fig.update_yaxes(title_text="Points")